## Daten via Zefix
Als klar wurde, dass ich mit den Adressen offenbar nicht alles extrahieren kann, habe ich aufs Zefix zurückgegriffen und gesehen, dass ich dort besser rankomme. Ich habe es mit dem Code unseres Unterrichts gemacht, um die Sache noch zu Ende zu bringen.

## Imports

In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile # für den Tor-Browser
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary # für den Tor-Browser
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re
from stem import Signal # Um beim Tor-Surfer die IP/Idendity zu wechseln
from stem.control import Controller #Um beim Tor-Surfer die IP/Idendity zu wechseln
from selenium.webdriver.support.ui import WebDriverWait # Um die Seite ganz zu laden brauchte ich aber doch nicht.
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common import exceptions 
from selenium.common.exceptions import ElementNotInteractableException
from random import randint #für random time sleep
from time import sleep #für random time sleep 
folderpath=r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg"

## Zefix Seiten besuchen und HTML-Code ablegen

###  Aus pragmatischen überlegungen entscheide ich mich für halbautomatische Abfrage
Denn ich konnte auf die Schnelle die  Eingabe der Gemeinde nicht automatisieren. Da der Kanton glücklicherweise nur 11 Gemeinden hat, kann man dies auch manuell lösen.
- Ich starte zu beginn mit Seleium den Browser und hole die Seite
- Danach werden Einstellungen manuell gemacht, sowie auch die jeweilige Gemeinde manuell eingegeben
- Danach geht es mit Selenium durch die Unterseiten und speichert jede ab.
- Hier finde ich Firmennamen (brauche ich gar (noch) nicht) sowie UID

In [ ]:
#Wir starten den Browser auf
driver = webdriver.Chrome(executable_path='C:\chromedriver.exe') #bei Windows ist das anders
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://www.zefix.ch')
#Geben wir ihm etwas ZEit


Hier dann von Hand die richtigen SAchen eingben
Bisher Zug, Baar, Cham, Hünenberg, Risch, Steinhausen, Walchwil, Unterägeri,Oberägeri, Menzingen, Neuheim

In [ ]:

gemeinde="Neuheim"
page = driver.page_source.encode('utf-8')
with open(folderpath+"/Zefix/page"+str(gemeinde)+"first"+".htm", "wb+") as file:
    file.write(page)
    file.close()
#its the 14th page
for elem in range(4):
    driver.find_elements_by_class_name('btn')[14].click()
    time.sleep(3)
    page = driver.page_source.encode('utf-8')
    with open(folderpath+"/Zefix/page"+str(gemeinde)+str(elem)+".htm", "wb+") as file:
        file.write(page)
        file.close()
        

# Jetzt auslesen
Ich will neben dem Namen und der Adresse (hier Sitz) auch die UID. Mit dieser ist möglicherweise eine Abfrage des Handelsregisterauszuges möglich

In [ ]:
alle_firmen_dok=[]
path=folderpath+"/Zefix/"
for page in tqdm(os.listdir(path)):
    file=open(path+page, 'r')
    text = file.read()
    h = BeautifulSoup(text, 'html.parser')
    zeilen = h.find_all("tr", {"class":"ng-scope"})
    for punkt in zeilen:
        firma = punkt.find_all("div")[0].text
        u_id=punkt.find_all("div")[2].text
        sitz = punkt.find_all("div")[4].text
        kanton = punkt.find_all("div")[5].text
        urls_firma = punkt.find_next("div").find_next("div").find("a")["href"]
    
        minidict_b={"Name":firma, "UID":u_id, "Sitz":sitz, "Kanton":kanton, "Webpage":urls_firma}
    
        alle_firmen_dok.append(minidict_b)

In [ ]:
dfalseit=pd.DataFrame(alle_firmen_dok)

In [ ]:
dfalseit.to_csv(folderpath+"/BRFrames/Zefix_erste_Seiten_ausgelesen")
dfalseit # Jeeeeessss es geht so:-)


In [ ]:
df1

In [ ]:
dfalseit=pd.read_csv(folderpath+"/BRFrames/Zefix_erste_Seiten_ausgelesen")
dfalseit

In [ ]:
# Hier noch formatieren, um die UID im richtigen Format zu haben

In [ ]:
dfalseit["UID"]=dfalseit["UID"].str.replace(".","")

In [ ]:
dfalseit["UID"]=dfalseit["UID"].str.replace("-","")

In [ ]:
dfalseit

## Jetzt URLs durchgehen und die Seiten abspeichern. 

### Als erstes der Versuch mit dem Zuger Handesregister, wo ich den ganzen Auszug abspeichern könnte
Hoffentlich ist hierbei das Zuger Handelsregister etwas netter:-)
-> funktioniert nicht, könnte aber durch Anpassungen doch noch funktionieren. Müsste man noch tüfteln

In [ ]:
number = 0
for page in tqdm(df1["Webpage"]):
    page_content = requests.get(page)
    page_content = page_content.text
    with open(folderpath+"/UID_Seiten/firma"+str(number)+".html", "w") as file:
        file.write(page_content)
        file.close()
    number +=1

### Muss mit UID bei ZEFIX durch die Sache hindurch.
(Leider erhalte ich so als Zusatzinfo eigentlich nur die Adresse)
Ein Telefon bei der UID-Stelle hat nun ergeben, dass der Surfer nur 10 Abfragen pro Minute zulässt. Muss also die Anzahl Abrufe einschränken.

In [ ]:
path="https://www.uid.admin.ch/Detail.aspx?uid_id="
fileliste_uid=[]
number = 0
for uid in tqdm(dfalseit["UID"]):
    page_content = requests.get(path+uid)
    page_content = page_content.text
    with open(folderpath+"/UID_Seiten/firma"+str(number)+".html", "w") as file:
        file.write(page_content)
        file.close()
    number +=1
    time.sleep(randint(6,8))
    page="firma"+str(number)+".html"
    UID=uid
    
    minidict_uid={"UID_Abfrage":UID, "Seite_gespeichert":page}
    fileliste_uid.append(minidict_uid)
    


In [ ]:
df1=pd.DataFrame(fileliste_uid)
df1.to_csv(folderpath+"/BRFrames/UID_File_Liste")

# Auslesen und Geocode :
- siehe 301 ZEFIX Auslesen der Firmen-Files

# Telefonbuch API und auslesen: 
- siehe 302 ZEFIX fehlende Adresse durch Telsearch API

## Fehlende Zefix-Einträge nun zu einem funktionierenden Dataframe zusammenfügen. 
- Anzahl Firmen 
- Anzahl Telbucheinträge
- Adresse ohne PLZ
- Adresse mit PLZ
- Geokoordinaten (müssen vor dem Zusammenfügen mit dem ursprünglichen Frame noch in CH-Format umgewandelt werden - oder umgekehrt.

### Die Telefonbucheinträge

In [13]:
df_fehlendetel=pd.read_csv(folderpath+"/BRFrames/Adressen_alle3713_mit_Anzahl_telbucheintraegen_extrahiert.csv")
df_fehlendetel=df_fehlendetel[["Adresse PLZ", "Anzahl"]]# Frame auf das Wesentliche verkleinern
df_fehlendetel.columns=["Adresse", "Telbuch"] #hier noch umbenennen
df_fehlendetel

,Adresse,Telbuch
0,"Haldenstrasse 5, 6340 Baar",44
1,"Gewerbestrasse 5, 6330 Cham",51
2,"Neuhofstrasse 5A, 6340 Baar",34
3,"Zugerstrasse 76B, 6340 Baar",29
4,"Alte Steinhauserstrasse 1, 6330 Cham",82
...,...,...
3708,"Huobstrasse 81A, 6330 Cham",0
3709,"Huobstrasse 85A, 6333 Hünenberg See",1
3710,"Huobweid 2, 6333 Hünenberg See",1
3711,"Huobweid 3, 6333 Hünenberg See",7


### Die Geodaten pro Adresse

In [3]:
dflolaganz=pd.read_csv(folderpath+"/BRFrames/Adressen_alle3713_mit_geocode_einzeln.csv")
dflolaganz

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Adresse,Unnamed: 0.1.1.1.1,output,lat,lon
0,0,0,0,0,"Haldenstrasse 5, 6340 Baar",0.0,"'lat': 47.1960393, 'lng': 8.538261499999999",47.1960393,8.538261499999999
1,1,1,1,1,"Gewerbestrasse 5, 6330 Cham",1.0,"'lat': 47.1883571, 'lng': 8.4734099",47.1883571,8.4734099
2,2,2,2,2,"Neuhofstrasse 5A, 6340 Baar",2.0,"'lat': 47.1913654, 'lng': 8.5144487",47.1913654,8.5144487
3,3,3,3,3,"Zugerstrasse 76B, 6340 Baar",3.0,"'lat': 47.1832252, 'lng': 8.518616999999999",47.1832252,8.518616999999999
4,4,4,4,4,"Alte Steinhauserstrasse 1, 6330 Cham",4.0,"'lat': 47.1877443, 'lng': 8.4737621",47.1877443,8.4737621
...,...,...,...,...,...,...,...,...,...
3708,3708,3708,3708,1708,"Huobstrasse 81A, 6330 Cham",NaN,"'lat': 47.1785418, 'lng': 8.4439531",47.1785418,8.4439531
3709,3709,3709,3709,1709,"Huobstrasse 85A, 6333 Hünenberg See",NaN,"'lat': 47.1788289, 'lng': 8.444037699999999}, ...","47.1788289, 'lng': 8.444037699999999}, 'locat...","8.444037699999999}, 'location_type': 'ROOFTOP..."
3710,3710,3710,3710,1710,"Huobweid 2, 6333 Hünenberg See",NaN,"'lat': 47.1774752, 'lng': 8.445678",47.1774752,8.445678
3711,3711,3711,3711,1711,"Huobweid 3, 6333 Hünenberg See",NaN,"'lat': 47.1780711, 'lng': 8.445626299999999",47.1780711,8.445626299999999


### Die Anzahl Firmen pro Adresse

In [8]:
df_fehlende_adressen=pd.read_csv(folderpath+"/BRFrames/Adressen_ohne_geocode_plz.csv")
df_fehlende_adressen

,Adresse PLZ,Unnamed: 0_x,Unnamed: 0.1,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,GDENAME,...,Zusatz,c/o,Strasse,Ergänzung,PLZ/Ort,Gemeinde,EGID,Rechtsform,MWSt-Start,Ort
0,"Haldenstrasse 5, 6340 Baar",0,0,258,0,0,0,0,0,0,...,0,258,258,0,258,258,183,257,226,258
1,"Gewerbestrasse 5, 6330 Cham",0,0,218,0,0,0,0,0,0,...,0,218,218,0,218,218,153,211,199,218
2,"Neuhofstrasse 5A, 6340 Baar",0,0,216,0,0,0,0,0,0,...,0,216,216,0,216,216,166,155,157,216
3,"Zugerstrasse 76B, 6340 Baar",0,0,181,0,0,0,0,0,0,...,0,181,181,0,181,181,126,174,156,181
4,"Alte Steinhauserstrasse 1, 6330 Cham",0,0,179,0,0,0,0,0,0,...,0,179,179,0,179,179,149,169,166,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3708,"Huobstrasse 81A, 6330 Cham",0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,1,1,0,0,1
3709,"Huobstrasse 85A, 6333 Hünenberg See",0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,1
3710,"Huobweid 2, 6333 Hünenberg See",0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,1,1
3711,"Huobweid 3, 6333 Hünenberg See",0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,1


In [9]:
df_firmen=df_fehlende_adressen[["Name", "Adresse PLZ"]]  
df_firmen.columns=["Firmen", "Adresse"] #umbenennen zum zusammenfügen

In [10]:
df_geoko=dflolaganz[["Adresse", "lat", "lon"]] #df nur mit lon und lat sowie Adresse
df_geoko

,Adresse,lat,lon
0,"Haldenstrasse 5, 6340 Baar",47.1960393,8.538261499999999
1,"Gewerbestrasse 5, 6330 Cham",47.1883571,8.4734099
2,"Neuhofstrasse 5A, 6340 Baar",47.1913654,8.5144487
3,"Zugerstrasse 76B, 6340 Baar",47.1832252,8.518616999999999
4,"Alte Steinhauserstrasse 1, 6330 Cham",47.1877443,8.4737621
...,...,...,...
3708,"Huobstrasse 81A, 6330 Cham",47.1785418,8.4439531
3709,"Huobstrasse 85A, 6333 Hünenberg See","47.1788289, 'lng': 8.444037699999999}, 'locat...","8.444037699999999}, 'location_type': 'ROOFTOP..."
3710,"Huobweid 2, 6333 Hünenberg See",47.1774752,8.445678
3711,"Huobweid 3, 6333 Hünenberg See",47.1780711,8.445626299999999


## Die Adressen mit den Geodaten und Telefonbucheinträgen zusammenfügen

In [11]:
df_alle_erg=pd.merge(df_geoko, df_firmen, on="Adresse", how="inner")

In [14]:
df_alle_erg_tot=pd.merge(df_alle_erg, df_fehlendetel, on="Adresse", how="inner")

### Kontrolle der Daten

In [15]:
df_extrac_sauber=df_alle_erg_tot[df_alle_erg_tot["lat"].str.contains("locat")==False]
df_extrac_sauber.sort_values("Firmen", ascending=False) 

,Adresse,lat,lon,Firmen,Telbuch
0,"Haldenstrasse 5, 6340 Baar",47.1960393,8.538261499999999,258,44
1,"Gewerbestrasse 5, 6330 Cham",47.1883571,8.4734099,218,51
2,"Neuhofstrasse 5A, 6340 Baar",47.1913654,8.5144487,216,34
3,"Zugerstrasse 76B, 6340 Baar",47.1832252,8.518616999999999,181,29
4,"Alte Steinhauserstrasse 1, 6330 Cham",47.1877443,8.4737621,179,82
...,...,...,...,...,...
2224,"Underbachstrasse 3, 6318 Walchwil",47.0983873,8.5196983,1,1
2223,"Twärfallenstrasse 3, 6313 Finstersee",47.1666815,8.629589,1,1
2222,"Rigiweg 17A, 634304 Holzhäusern ZG",47.1551432,8.4389377,1,0
2221,"Rigiweg 21A, 6343 Holzhäusern ZG",47.1545303,8.4385846,1,0


In [9]:
# HIer alle, die noch nciht richtig geortet sind. 283... da muss ich nochmals drüber.
df_nichtextr=df_alle_erg_tot[df_alle_erg_tot["lat"].str.contains("locat")==True]
df_nichtextr

,Adresse,lat,lon,Firmen,Telbuch
41,"Blegistrasse 9, 6340 Baar","47.2159991, 'lng': 8.570899599999999}, 'locat...","8.570899599999999}, 'location_type': 'ROOFTOP...",43,14
50,"Eichmattstrasse 97, 6333 Hünenberg See","47.1792367, 'lng': 8.443230999999999}, 'locat...","8.443230999999999}, 'location_type': 'ROOFTOP...",34,4
63,"Bahnhofstrasse 9, 6341 Baar","47.3681441, 'lng': 8.5393976}, 'location_type...","8.5393976}, 'location_type': 'ROOFTOP', 'view...",24,2
97,"Luzernerstrasse 72, 6333 Hünenberg See","47.1754141, 'lng': 8.4508203}, 'location_type...","8.4508203}, 'location_type': 'ROOFTOP', 'view...",15,9
122,"Oberneuhofstrasse 5, 6341 Baar","47.1890701, 'lng': 8.5164049}, 'location_type...","8.5164049}, 'location_type': 'ROOFTOP', 'view...",12,1
...,...,...,...,...,...
3734,"Huobrain 5, 6333 Hünenberg See","47.17794670000001, 'lng': 8.4419982}, 'locati...","8.4419982}, 'location_type': 'ROOFTOP', 'view...",1,1
3737,"Huobstrasse 21, 6333 Hünenberg See","47.1755407, 'lng': 8.4452193}, 'location_type...","8.4452193}, 'location_type': 'ROOFTOP', 'view...",1,0
3738,"Huobstrasse 3, 6333 Hünenberg See","47.1739298, 'lng': 8.4478046}, 'location_type...","8.4478046}, 'location_type': 'ROOFTOP', 'view...",1,7
3739,"Huobstrasse 77B, 6333 Hünenberg See","47.1782182, 'lng': 8.4439207}, 'location_type...","8.4439207}, 'location_type': 'ROOFTOP', 'view...",1,1


In [14]:
df_nichtextr["lat1"]=df_nichtextr["lat"].str.extract("(\d*\.\d*)\,.\'lng") # Ändere Regex um zu extrahieren

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_nichtextr

,Adresse,lat,lon,Firmen,Telbuch,lat1
41,"Blegistrasse 9, 6340 Baar","47.2159991, 'lng': 8.570899599999999}, 'locat...","8.570899599999999}, 'location_type': 'ROOFTOP...",43,14,47.2159991
50,"Eichmattstrasse 97, 6333 Hünenberg See","47.1792367, 'lng': 8.443230999999999}, 'locat...","8.443230999999999}, 'location_type': 'ROOFTOP...",34,4,47.1792367
63,"Bahnhofstrasse 9, 6341 Baar","47.3681441, 'lng': 8.5393976}, 'location_type...","8.5393976}, 'location_type': 'ROOFTOP', 'view...",24,2,47.3681441
97,"Luzernerstrasse 72, 6333 Hünenberg See","47.1754141, 'lng': 8.4508203}, 'location_type...","8.4508203}, 'location_type': 'ROOFTOP', 'view...",15,9,47.1754141
122,"Oberneuhofstrasse 5, 6341 Baar","47.1890701, 'lng': 8.5164049}, 'location_type...","8.5164049}, 'location_type': 'ROOFTOP', 'view...",12,1,47.1890701
...,...,...,...,...,...,...
3734,"Huobrain 5, 6333 Hünenberg See","47.17794670000001, 'lng': 8.4419982}, 'locati...","8.4419982}, 'location_type': 'ROOFTOP', 'view...",1,1,47.17794670000001
3737,"Huobstrasse 21, 6333 Hünenberg See","47.1755407, 'lng': 8.4452193}, 'location_type...","8.4452193}, 'location_type': 'ROOFTOP', 'view...",1,0,47.1755407
3738,"Huobstrasse 3, 6333 Hünenberg See","47.1739298, 'lng': 8.4478046}, 'location_type...","8.4478046}, 'location_type': 'ROOFTOP', 'view...",1,7,47.1739298
3739,"Huobstrasse 77B, 6333 Hünenberg See","47.1782182, 'lng': 8.4439207}, 'location_type...","8.4439207}, 'location_type': 'ROOFTOP', 'view...",1,1,47.1782182


In [16]:
df_nichtextr["lon1"]=df_nichtextr["lat"].str.extract("(\d\.\d*)\}\,") 

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
# Nun jene spalten nehmen und umbenennen um sie wieder zusammenzuführen
df_nichtextr_neu=df_nichtextr[["Adresse", "lat1", "lon1","Firmen", "Telbuch"]]
df_nichtextr_neu.columns=["Adresse", "lat", "lon", "Firmen", "Telbuch"]
df_nichtextr_neu

,Adresse,lat,lon,Firmen,Telbuch
41,"Blegistrasse 9, 6340 Baar",47.2159991,8.570899599999999,43,14
50,"Eichmattstrasse 97, 6333 Hünenberg See",47.1792367,8.443230999999999,34,4
63,"Bahnhofstrasse 9, 6341 Baar",47.3681441,8.5393976,24,2
97,"Luzernerstrasse 72, 6333 Hünenberg See",47.1754141,8.4508203,15,9
122,"Oberneuhofstrasse 5, 6341 Baar",47.1890701,8.5164049,12,1
...,...,...,...,...,...
3734,"Huobrain 5, 6333 Hünenberg See",47.17794670000001,8.4419982,1,1
3737,"Huobstrasse 21, 6333 Hünenberg See",47.1755407,8.4452193,1,0
3738,"Huobstrasse 3, 6333 Hünenberg See",47.1739298,8.4478046,1,7
3739,"Huobstrasse 77B, 6333 Hünenberg See",47.1782182,8.4439207,1,1


In [28]:
df_sauber_fehlende_adressen=pd.concat([df_nichtextr_neu, df_extrac_sauber])
df_sauber_fehlende_adressen.to_csv(folderpath+"/BRFrames/Adressen_alle3713_mit_telbuch_geocode_sauber.csv")
df_sauber_fehlende_adressen

,Adresse,lat,lon,Firmen,Telbuch
41,"Blegistrasse 9, 6340 Baar",47.2159991,8.570899599999999,43,14
50,"Eichmattstrasse 97, 6333 Hünenberg See",47.1792367,8.443230999999999,34,4
63,"Bahnhofstrasse 9, 6341 Baar",47.3681441,8.5393976,24,2
97,"Luzernerstrasse 72, 6333 Hünenberg See",47.1754141,8.4508203,15,9
122,"Oberneuhofstrasse 5, 6341 Baar",47.1890701,8.5164049,12,1
...,...,...,...,...,...
3736,"Huobstrasse 15B, 6331 Hünenberg",47.1752779,8.4456597,1,0
3740,"Huobstrasse 81A, 6330 Cham",47.1785418,8.4439531,1,0
3742,"Huobweid 2, 6333 Hünenberg See",47.1774752,8.445678,1,1
3743,"Huobweid 3, 6333 Hünenberg See",47.1780711,8.445626299999999,1,7


# Nächste Schritte:
- Vergleichswert erstellen
- Normale Adressen hinzufügen um mit anderem Frame zusammenzubringen
- Geodataframe erstellen und Geocode ändern
- Beide Frames zusammenbringen. (mit unterschiedlichen Geocodes)

In [2]:
df_sauber_fehlende_adressen=pd.read_csv(folderpath+"/BRFrames/Adressen_alle3713_mit_telbuch_geocode_sauber.csv")

In [3]:
df_alles=pd.read_csv(folderpath+"/BRFrames/Kanton_ganz_mit_Vergleichswert")

In [4]:
df_alles

,Unnamed: 0,Unnamed: 0.1,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,GDENAME,GKODE,GKODN,Vergleich
0,0,0,"Zugerstrasse 74, Baar",164,frontpage2_976.htm,12,telfront10000.htm,Zugerstrasse,Baar,2681933.977,1226416.568,13.666667
1,1,1,"Mühlegasse 18, Baar",92,frontpage30_262.htm,57,telfront10001.htm,Mühlegasse,Baar,2682425.299,1228159.836,1.614035
2,2,2,"Blegistrasse 11b, Baar",88,frontpage2_936.htm,13,telfront10002.htm,Blegistrasse,Baar,2685739.840,1230044.629,6.769231
3,3,3,"Sihlbruggstrasse 105, Baar",80,frontpage2_530.htm,39,telfront10003.htm,Sihlbruggstrasse,Baar,2684903.579,1229684.742,2.051282
4,4,4,"Ruessenstrasse 12, Baar",75,frontpage2_963.htm,16,telfront10004.htm,Ruessenstrasse,Baar,2685107.750,1229580.996,4.687500
...,...,...,...,...,...,...,...,...,...,...,...,...
6975,4684,4685,"Chamerstrasse 51.z, Zug",0,frontpage1032.htm,1,telfront995.htm,Chamerstrasse,Zug,2680782.000,1225353.000,0.000000
6976,4685,4686,"Chamerstrasse 52, Zug",11,frontpage1033.htm,5,telfront996.htm,Chamerstrasse,Zug,2680846.193,1225345.734,2.200000
6977,4686,4687,"Chamerstrasse 53, Zug",0,frontpage1034.htm,1,telfront997.htm,Chamerstrasse,Zug,2680773.323,1225372.849,0.000000
6978,4687,4688,"Chamerstrasse 54, Zug",3,frontpage1035.htm,4,telfront998.htm,Chamerstrasse,Zug,2680828.182,1225361.153,0.750000


In [5]:
df_sauber_fehlende_adressen

,Unnamed: 0,Adresse,lat,lon,Firmen,Telbuch
0,41,"Blegistrasse 9, 6340 Baar",47.215999,8.570900,43,14
1,50,"Eichmattstrasse 97, 6333 Hünenberg See",47.179237,8.443231,34,4
2,63,"Bahnhofstrasse 9, 6341 Baar",47.368144,8.539398,24,2
3,97,"Luzernerstrasse 72, 6333 Hünenberg See",47.175414,8.450820,15,9
4,122,"Oberneuhofstrasse 5, 6341 Baar",47.189070,8.516405,12,1
...,...,...,...,...,...,...
3713,3736,"Huobstrasse 15B, 6331 Hünenberg",47.175278,8.445660,1,0
3714,3740,"Huobstrasse 81A, 6330 Cham",47.178542,8.443953,1,0
3715,3742,"Huobweid 2, 6333 Hünenberg See",47.177475,8.445678,1,1
3716,3743,"Huobweid 3, 6333 Hünenberg See",47.178071,8.445626,1,7


In [11]:
#Adressformat gleich gestalten
df_sauber_fehlende_adressen["Adresse"]=df_sauber_fehlende_adressen["Adresse"].str.replace("\d\d\d\d","")

In [13]:
df_sauber_fehlende_adressen["STRNAME"]=df_sauber_fehlende_adressen["Adresse"].str.extract("(\w*)..*")

In [15]:
df_sauber_fehlende_adressen.to_csv(folderpath+"/BRFrames/Adressen_alle3713_mit_telbuch_geocode_angepasst_ursprung_df.csv")